In [1]:
import glob
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os

2025-02-06 12:41:43.543175: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 12:41:43.565262: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-06 12:41:43.906592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Check if GPU is available
if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU not found, using CPU instead")

GPU is available


2025-02-06 12:41:45.971029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-06 12:41:45.990266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-06 12:41:45.994855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [4]:
data_path = r'/home/fede/PhD/UNIGE/Dataset_UniCa/Cagliari/Dataset'

Glob_data = glob.glob(data_path + "/*/*")

class Dataset:
    def __init__(self, base_file_path):
        # Initialize with a base file path, set a maximum number of files per class, and initialize variables for data
        self.max_files_per_class = 50
        self.base_file_path = base_file_path
        self.X = []
        self.y = []
        self.class_counts = {}
        self.text_files = self._get_text_files()
        
    def _get_text_files(self):
        # Get all directories in the base file path
        data = glob.glob(self.base_file_path + "/*")
        category_names = [os.path.basename(directory_path) for directory_path in data]
        text_files = {category: [] for category in category_names}
        
        # Map each category to its corresponding directories
        for directory_path in data:
            category = os.path.basename(directory_path)
            if category in text_files:
                text_files[category].append(directory_path)
        
        return text_files
    
    def _load_data(self):
        # Load data from each file into X and y arrays
        for label, (key, directory_paths) in enumerate(self.text_files.items(), start=0):
            self.class_counts[key] = 0
            for directory_path in directory_paths:
                files_in_directory = glob.glob(directory_path + "/*.txt")  # Looking for .txt files specifically
                for file_path in files_in_directory:
                    df = pd.read_csv(file_path, header=None,delimiter='\t', skiprows=1)
                    
                    df= df.iloc[4000:-4000, :]
                      # Extract specific part of the dataframe
                    scaler = StandardScaler()
                    df_normalized = scaler.fit_transform(df)
                    
                    # Randomly select three windows of 300 samples
                    for _ in range(10):
                        start_idx = random.randint(0, df_normalized.shape[0] - 300)
                        df_extract = df_normalized[start_idx:start_idx + 300, :]
                        # Append to the corresponding list based on the Text category
                        self.X.append(df_extract)
                        self.y.append(int(label))  
                        self.class_counts[key] += 1  # Update class count  

                  
                print(f"Class {key}: {self.class_counts[key]} files")
    
    def _prepare_data(self):
        # Convert X and y to numpy arrays and ensure correct data types
        self.X = np.asarray(self.X, dtype=np.float32)
        self.y = np.asarray(self.y, dtype=np.int32)
        print("X shape:", self.X.shape)
        print("y shape:", self.y.shape)
        
    def get_data_splits(self):
        # Load data, prepare it, and split it into training, validation, and test sets
        self._load_data()
        self._prepare_data()
        y_categorical = tf.keras.utils.to_categorical(self.y)
        X_train, x_tmp, y_train, y_tmp = train_test_split(self.X, y_categorical, test_size=0.3, random_state=42, stratify=self.y)
        X_val, X_test, y_val, y_test = train_test_split(x_tmp, y_tmp, test_size=0.5, random_state=42, stratify=np.argmax(y_tmp, axis=1))
        return X_train, y_train, X_val, y_val, X_test, y_test
    
X_train, y_train, X_val, y_val, X_test, y_test = Dataset(data_path).get_data_splits()

with open ('dataset.npy','wb') as f:
    np.save(f, X_train)
    np.save(f, y_train)
    np.save(f, X_val)
    np.save(f, y_val)
    np.save(f, X_test)
    np.save(f, y_test)

Class Text8: 4300 files
Class Text5: 4300 files
Class Text6: 4300 files
Class Text7: 4300 files
Class Text4: 4300 files
Class Text3: 2380 files
X shape: (23880, 300, 8)
y shape: (23880,)


In [25]:
import glob
import os
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

class Dataset:
    def __init__(self, base_file_path):
        # Initialize with a base file path, set a maximum number of files per class, and initialize variables for data
        self.max_files_per_class = 50
        self.base_file_path = base_file_path
        self.X = []
        self.y = []
        self.class_counts = {}
        self.text_files = self._get_text_files()
        
    def _get_text_files(self):
        # Get all directories in the base file path
        data = glob.glob(self.base_file_path + "/*")
        category_names = [os.path.basename(directory_path) for directory_path in data]
        text_files = {category: [] for category in category_names}
        
        # Map each category to its corresponding directories
        for directory_path in data:
            category = os.path.basename(directory_path)
            if category in text_files:
                text_files[category].append(directory_path)
        
        return text_files
    
    def _load_data(self):
        # Load data from each file into X and y arrays
        for label, (key, directory_paths) in enumerate(self.text_files.items(), start=0):
            self.class_counts[key] = 0
            for directory_path in directory_paths:
                files_in_directory = glob.glob(directory_path + "/*.txt")  # Looking for .txt files specifically
                for file_path in files_in_directory:
                    df = pd.read_csv(file_path, header=None, delimiter='\t', skiprows=1)
                    
                    df = df.iloc[4000:-4000, :]  # Extract specific part of the dataframe
                    scaler = StandardScaler()
                    df_normalized = scaler.fit_transform(df)
                    
                    # Randomly select three windows of 300 samples
                    for _ in range(10):
                        start_idx = random.randint(0, df_normalized.shape[0] - 300)
                        df_extract = df_normalized[start_idx:start_idx + 300, :]
                        # Append to the corresponding list based on the Text category
                        self.X.append(df_extract)
                        self.y.append(int(label))  
                        self.class_counts[key] += 1  # Update class count  

                print(f"Class {key}: {self.class_counts[key]} files")
    
    def _prepare_data(self):
        # Convert X and y to numpy arrays and ensure correct data types
        self.X = np.asarray(self.X, dtype=np.float32)
        self.y = np.asarray(self.y, dtype=np.int32)
        print("X shape:", self.X.shape)
        print("y shape:", self.y.shape)
        
    def get_data_splits(self):
        # Load data, prepare it, and split it into training, validation, and test sets
        self._load_data()
        self._prepare_data()
        
        # Convert y labels to one-hot encoding
        y_categorical = tf.keras.utils.to_categorical(self.y, num_classes=8)  # One-hot encoding
        
        # Split the dataset
        X_train, x_tmp, y_train, y_tmp = train_test_split(self.X, y_categorical, test_size=0.3, random_state=42, stratify=self.y)
        X_val, X_test, y_val, y_test = train_test_split(x_tmp, y_tmp, test_size=0.5, random_state=42, stratify=np.argmax(y_tmp, axis=1))
        
        # Return the training, validation, and test splits
        return X_train, y_train, X_val, y_val, X_test, y_test
    
# Initialize and process the dataset
data_path = r'/home/fede/PhD/UNIGE/Dataset_UniCa/Cagliari/Dataset'
X_train, y_train, X_val, y_val, X_test, y_test = Dataset(data_path).get_data_splits()

# Save the processed dataset to file
with open('dataset.npy', 'wb') as f:
    np.save(f, X_train)
    np.save(f, y_train)
    np.save(f, X_val)
    np.save(f, y_val)
    np.save(f, X_test)
    np.save(f, y_test)

# Optional: Print the shapes of the data to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


Class Text8: 4300 files
Class Text5: 4300 files
Class Text6: 4300 files
Class Text7: 4300 files
Class Text4: 4300 files
Class Text3: 2380 files
X shape: (23880, 300, 8)
y shape: (23880,)
X_train shape: (16716, 300, 8)
y_train shape: (16716, 8)
X_val shape: (3582, 300, 8)
y_val shape: (3582, 8)
X_test shape: (3582, 300, 8)
y_test shape: (3582, 8)
